In [99]:
import tensorflow as tf
tf.config.run_functions_eagerly(True)
from tensorflow.keras import layers,models,metrics
import matplotlib.pyplot as plt
import numpy as np
import os
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [100]:
CNNmodel = models.Sequential()
CNNmodel.add(layers.Conv2D(128,(5,5),strides = 1, activation ="relu",padding = "valid" ,input_shape=(17,17,18)))
CNNmodel.add(layers.Conv2D(128,(3,3),strides =1,padding = "same",activation = "relu",use_bias = 1))
CNNmodel.add(layers.Conv2D(128,(3,3),strides =1,padding = "same",activation = "relu",use_bias = 1))
CNNmodel.add(layers.Conv2D(128,(3,3),strides =1,padding = "same",activation = "relu",use_bias = 1))
CNNmodel.add(layers.Conv2D(128,(3,3),strides =1,padding = "same",activation = "relu",use_bias = 1))
CNNmodel.add(layers.Conv2D(128,(3,3),strides =1,padding = "same",activation = "relu",use_bias = 1))
CNNmodel.add(layers.Conv2D(128,(3,3),strides =1,padding = "same",activation = "relu",use_bias = 1))
CNNmodel.add(layers.Conv2D(128,(3,3),strides =1,padding = "same",activation = "relu",use_bias = 1))
CNNmodel.add(layers.Conv2D(1,(1,1),strides = 1))
CNNmodel.add(layers.Flatten())

In [101]:
CNNmodel.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_72 (Conv2D)          (None, 13, 13, 128)       57728     
                                                                 
 conv2d_73 (Conv2D)          (None, 13, 13, 128)       147584    
                                                                 
 conv2d_74 (Conv2D)          (None, 13, 13, 128)       147584    
                                                                 
 conv2d_75 (Conv2D)          (None, 13, 13, 128)       147584    
                                                                 
 conv2d_76 (Conv2D)          (None, 13, 13, 128)       147584    
                                                                 
 conv2d_77 (Conv2D)          (None, 13, 13, 128)       147584    
                                                                 
 conv2d_78 (Conv2D)          (None, 13, 13, 128)      

In [102]:
data = np.load('data/scoredPositionsFull.npz')
actionData = np.load('data/actions.npz')
positions = data['positions']
positions = positions.transpose(0,2,3,1)
actions = actionData['actions']
actions = np.delete(actions,0,1)
actions = np.delete(actions,0,1)
actions = np.delete(actions,14,1)
actions = np.delete(actions,13,1)
actions = np.delete(actions,0,2)
actions = np.delete(actions,0,2)
actions = np.delete(actions,14,2)
actions = np.delete(actions,13,2)
actions = np.reshape(actions,(539998,169))
actions = np.where(actions == 1)
actions = actions[1]
print (actions)

[150  48 107 ... 158 157   9]


In [103]:
CNNmodel.compile(optimizer = 'adam', loss = tf.keras.losses.SparseCategoricalCrossentropy(),metrics = [metrics.SparseCategoricalAccuracy()])

In [104]:
checkpoint_path = "training/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath = checkpoint_path,verbose = 1, save_weights_only = True, save_freq = 5*128)

In [105]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

In [107]:
import datetime
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [108]:
positions_test = positions[-30000:]
actions_test = actions[-30000:]
positions_train = positions[:-30000]
actions_train = actions[:-30000]

In [109]:
trained = CNNmodel.fit(positions_train,actions_train,batch_size = 128, epochs = 50,validation_data= (positions_test,actions_test),callbacks= [cp_callback,callback,tensorboard_callback])

Epoch 1/20
24/24 [==============================] - 35s 1s/step - loss: 5.2380 - sparse_categorical_accuracy: 0.0140 - val_loss: 5.1299 - val_sparse_categorical_accuracy: 6.6667e-04
Epoch 2/20
24/24 [==============================] - 32s 1s/step - loss: 5.1299 - sparse_categorical_accuracy: 3.3333e-04 - val_loss: 5.1299 - val_sparse_categorical_accuracy: 0.0010
Epoch 3/20
24/24 [==============================] - 33s 1s/step - loss: 5.1299 - sparse_categorical_accuracy: 3.3333e-04 - val_loss: 5.1299 - val_sparse_categorical_accuracy: 0.0010
Epoch 4/20
24/24 [==============================] - 33s 1s/step - loss: 5.1299 - sparse_categorical_accuracy: 3.3333e-04 - val_loss: 5.1299 - val_sparse_categorical_accuracy: 0.0010
Epoch 5/20
24/24 [==============================] - 32s 1s/step - loss: 5.1299 - sparse_categorical_accuracy: 3.3333e-04 - val_loss: 5.1299 - val_sparse_categorical_accuracy: 0.0010
